# Try and get coordinates for nstates in prior cov matrix

In [1]:
from datetime import date
from nearestPD import nearestPD
from functions import points2distance, find_centre_region
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

region = xr.open_dataset("regions_verify_202011.nc")
region

<xarray.Dataset>
Dimensions:             (latitude: 180, longitude: 360)
Coordinates:
  * latitude            (latitude) float64 -89.5 -88.5 -87.5 ... 87.5 88.5 89.5
  * longitude           (longitude) float64 -179.5 -178.5 -177.5 ... 178.5 179.5
Data variables:
    transcom_regions    (latitude, longitude) int32 ...
    regions_bio         (latitude, longitude) int64 ...
    regions_bio_categ   (latitude, longitude) int64 ...
    regions_anth        (latitude, longitude) int64 ...
    regions_anth_categ  (latitude, longitude) int64 ...
Attributes:
    description:  Covariance related parameters for CTDAS-CH4

Idea: modify create_covariance code to loop through regions outside nhl, i.e. categories 1-26 and get corresponding tc so that can use the find_centre_region function

In [2]:
def gettc(region, j=None):
    if j == None:
        tc = np.array(list(set(region.transcom_regions.values.flatten())))
        tc = tc[~np.isnan(tc)]
    else:
        w = np.where(region['regions_%s' % v].values == j)
        tc = list(set(region.transcom_regions.values[w].flatten()))
        if len(tc) != 1:
            print('[ERROR] len(tc) not equal to 1!')
            sys.exit()
    tc = int(tc[0])
    return tc

In [3]:
nr = region['regions_bio'].values.max() #6858
    
categ = region['regions_bio_categ']

for i in [1]:# set(categ.values.flatten()):
    print('  categ: ', i)
    # filter variables in dataset "region" so that everything for which categ not i is set to nan
    dummy = region.where(categ == i, drop=True)
    # get tc corresponding to categ i
    tc = gettc(dummy, None)
    print(tc)
    #dregions = dummy['regions_bio'].values.flatten()
    #dregions = set(dregions[~np.isnan(dregions)])

  categ:  1
7


In [4]:
nhl = list(range(1, 7)) + [15, 16] + list(range(23, 30))

regions_bio = region["regions_bio"]
nr = regions_bio.values.max
#find tc corresponding to regions_bio

for i in range(nr):
    dummy = region.where(regions_bio == i, drop=True )
    tc = gettc(dummy, None)
    if tc in nhl:
        continue #these calculated differently
    else:
        centre = find_centre_region()

6
